Basic imports and path management

In [ ]:
import pathlib
import sys
import time
import glob
from tqdm import tqdm
import multiprocessing

root = pathlib.Path("../../").resolve()
sys.path.append(str(root.joinpath("src", "python_framework_v02")))
sys.path.append(str(root.joinpath("src", "python_framework_v01")))
sys.path.append(str(root.joinpath("src", "python_routing_v01")))
sys.path.append(".")
import nhd_io as nio
import compute_nhd_routing_SingleSeg as tr
import nhd_network_utilities_v01 as nnu
import nhd_reach_utilities as nru

custom_input_folder = root.joinpath("test", "input", "yaml")
custom_input_file = "florence_933020089_dt60.yaml"
run_pocono2_test = None


Read the primary data input from json

In [ ]:
supernetwork_parameters = None
waterbody_parameters = None
if custom_input_file:
    (
        supernetwork_parameters,
        waterbody_parameters,
        forcing_parameters,
        restart_parameters,
        output_parameters,
        run_parameters,
    ) = nio.read_custom_input(custom_input_folder.joinpath(custom_input_file))

    break_network_at_waterbodies = run_parameters.get(
        "break_network_at_waterbodies", None
    )

    dt = run_parameters.get("dt", None)
    nts = run_parameters.get("nts", None)
    qts_subdivisions = run_parameters.get("qts_subdivisions", None)
    debuglevel = -1 * int(run_parameters.get("debuglevel", 0))
    verbose = run_parameters.get("verbose", None)
    showtiming = run_parameters.get("showtiming", None)
    percentage_complete = run_parameters.get("percentage_complete", None)
    do_network_analysis_only = run_parameters.get("do_network_analysis_only", None)
    assume_short_ts = run_parameters.get("assume_short_ts", None)
    parallel_compute = run_parameters.get("parallel_compute", None)
    cpu_pool = run_parameters.get("cpu_pool", None)
    sort_networks = run_parameters.get("sort_networks", None)

    csv_output = output_parameters.get("csv_output", None)
    nc_output_folder = output_parameters.get("nc_output_folder", None)

    qlat_const = forcing_parameters.get("qlat_const", None)
    qlat_input_file = forcing_parameters.get("qlat_input_file", None)
    qlat_input_folder = forcing_parameters.get("qlat_input_folder", None)
    qlat_file_pattern_filter = forcing_parameters.get("qlat_file_pattern_filter", None)
    qlat_file_index_col = forcing_parameters.get("qlat_file_index_col", None)
    qlat_file_value_col = forcing_parameters.get("qlat_file_value_col", None)

    wrf_hydro_channel_restart_file = restart_parameters.get(
        "wrf_hydro_channel_restart_file", None
    )
    wrf_hydro_channel_ID_crosswalk_file = restart_parameters.get(
        "wrf_hydro_channel_ID_crosswalk_file", None
    )
    wrf_hydro_channel_ID_crosswalk_file_field_name = restart_parameters.get(
        "wrf_hydro_channel_ID_crosswalk_file_field_name", None
    )
    wrf_hydro_channel_restart_upstream_flow_field_name = restart_parameters.get(
        "wrf_hydro_channel_restart_upstream_flow_field_name", None
    )
    wrf_hydro_channel_restart_downstream_flow_field_name = restart_parameters.get(
        "wrf_hydro_channel_restart_downstream_flow_field_name", None
    )
    wrf_hydro_channel_restart_depth_flow_field_name = restart_parameters.get(
        "wrf_hydro_channel_restart_depth_flow_field_name", None
    )

    wrf_hydro_waterbody_restart_file = restart_parameters.get(
        "wrf_hydro_waterbody_restart_file", None
    )
    wrf_hydro_waterbody_ID_crosswalk_file = restart_parameters.get(
        "wrf_hydro_waterbody_ID_crosswalk_file", None
    )
    wrf_hydro_waterbody_ID_crosswalk_file_field_name = restart_parameters.get(
        "wrf_hydro_waterbody_ID_crosswalk_file_field_name", None
    )
    wrf_hydro_waterbody_crosswalk_filter_file = restart_parameters.get(
        "wrf_hydro_waterbody_crosswalk_filter_file", None
    )
    wrf_hydro_waterbody_crosswalk_filter_file_field_name = restart_parameters.get(
        "wrf_hydro_waterbody_crosswalk_filter_file_field_name", None
    )

# Any specific commandline arguments will override the file
# TODO: There are probably some pathological collisions that could
# arise from this ordering ... check these out.


In [ ]:
if run_pocono2_test:
    if verbose:
        print("running test case for Pocono_TEST2 domain")
    # Overwrite the following test defaults
    supernetwork = "Pocono_TEST2"
    break_network_at_waterbodies = False
    qts_subdivisions = 1  # change qts_subdivisions = 1 as  default
    dt = 300 / qts_subdivisions
    nts = 144 * qts_subdivisions
    csv_output = {"csv_output_folder": os.path.join(root, "test", "output", "text")}
    nc_output_folder = os.path.join(root, "test", "output", "text")
    # test 1. Take lateral flow from re-formatted wrf-hydro output from Pocono Basin simulation
    qlat_input_file = os.path.join(
        root, r"test/input/geo/PoconoSampleData2/Pocono_ql_testsamp1_nwm_mc.csv"
    )


In [ ]:
if showtiming:
    program_start_time = time.time()
if verbose:
    print(f"begin program t-route ...")

# STEP 1: Read the supernetwork dataset and build the connections graph
if verbose:
    print("creating supernetwork connections set")
if showtiming:
    start_time = time.time()

if supernetwork_parameters:
    supernetwork_values = nnu.get_nhd_connections(
        supernetwork_parameters=supernetwork_parameters,
        verbose=False,
        debuglevel=debuglevel,
    )

else:
    test_folder = os.path.join(root, r"test")
    geo_input_folder = os.path.join(test_folder, r"input", r"geo")
    supernetwork_parameters, supernetwork_values = nnu.set_networks(
        supernetwork=supernetwork,
        geo_input_folder=geo_input_folder,
        verbose=False,
        debuglevel=debuglevel,
    )
    waterbody_parameters = nnu.set_waterbody_parameters(
        supernetwork=supernetwork,
        geo_input_folder=geo_input_folder,
        verbose=False,
        debuglevel=debuglevel,
    )

if verbose:
    print("supernetwork connections set complete")
if showtiming:
    print("... in %s seconds." % (time.time() - start_time))

connections = supernetwork_values[0]


In [ ]:
# STEP 2: Separate the networks and build the sub-graph of reaches within each network
if showtiming:
    start_time = time.time()
if verbose:
    print("organizing connections into reaches ...")
networks = nru.compose_networks(
    supernetwork_values,
    break_network_at_waterbodies=break_network_at_waterbodies,
    verbose=False,
    debuglevel=debuglevel,
    showtiming=showtiming,
)

if verbose:
    print("reach organization complete")
if showtiming:
    print("... in %s seconds." % (time.time() - start_time))
    start_time = time.time()


In [ ]:
# STEP 3: Organize Network for Waterbodies
if break_network_at_waterbodies:
    if showtiming:
        start_time = time.time()
    if verbose:
        print("reading waterbody parameter file ...")

    ## STEP 3a: Read waterbody parameter file
    waterbodies_values = supernetwork_values[12]
    waterbodies_segments = supernetwork_values[13]
    connections_tailwaters = supernetwork_values[4]

    waterbodies_df = nio.read_waterbody_df(waterbody_parameters, waterbodies_values,)
    waterbodies_df = waterbodies_df.sort_index(axis="index").sort_index(axis="columns")

    nru.order_networks(connections, networks, connections_tailwaters)

    if verbose:
        print("waterbodies complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))
        start_time = time.time()

    ## STEP 3b: Order subnetworks above and below reservoirs
    if showtiming:
        start_time = time.time()
    if verbose:
        print("ordering waterbody subnetworks ...")

    max_network_seqorder = -1
    for network in networks:
        max_network_seqorder = max(
            networks[network]["network_seqorder"], max_network_seqorder
        )
    ordered_networks = {}

    for terminal_segment, network in networks.items():
        if network["network_seqorder"] not in ordered_networks:
            ordered_networks[network["network_seqorder"]] = []
        ordered_networks[network["network_seqorder"]].append(
            (terminal_segment, network)
        )

    if verbose:
        print("ordering waterbody subnetworks complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))
        start_time = time.time()

else:
    # If we are not splitting the networks, we can put them all in one order
    max_network_seqorder = 0
    ordered_networks = {}
    ordered_networks[0] = [
        (terminal_segment, network) for terminal_segment, network in networks.items()
    ]

if do_network_analysis_only:
    sys.exit()

if break_network_at_waterbodies:
    ## STEP 3c: Handle Waterbody Initial States
    if showtiming:
        start_time = time.time()
    if verbose:
        print("setting waterbody initial states ...")

    if wrf_hydro_waterbody_restart_file:

        waterbody_initial_states_df = nio.get_reservoir_restart_from_wrf_hydro(
            wrf_hydro_waterbody_restart_file,
            wrf_hydro_waterbody_ID_crosswalk_file,
            wrf_hydro_waterbody_ID_crosswalk_file_field_name,
            wrf_hydro_waterbody_crosswalk_filter_file,
            wrf_hydro_waterbody_crosswalk_filter_file_field_name,
        )
    else:
        # TODO: Consider adding option to read cold state from route-link file
        waterbody_initial_ds_flow_const = 0.0
        waterbody_initial_depth_const = 0.0
        # Set initial states from cold-state
        waterbody_initial_states_df = pd.DataFrame(
            0, index=waterbodies_df.index, columns=["qd0", "h0",], dtype="float32"
        )
        # TODO: This assignment could probably by done in the above call
        waterbody_initial_states_df["qd0"] = waterbody_initial_ds_flow_const
        waterbody_initial_states_df["h0"] = waterbody_initial_depth_const
        waterbody_initial_states_df["index"] = range(len(waterbody_initial_states_df))

    if verbose:
        print("waterbody initial states complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))
        start_time = time.time()


In [ ]:
# STEP 4: Handle Channel Initial States
if showtiming:
    start_time = time.time()
if verbose:
    print("setting channel initial states ...")

if wrf_hydro_channel_restart_file:

    channel_initial_states_df = nio.get_stream_restart_from_wrf_hydro(
        wrf_hydro_channel_restart_file,
        wrf_hydro_channel_ID_crosswalk_file,
        wrf_hydro_channel_ID_crosswalk_file_field_name,
        wrf_hydro_channel_restart_upstream_flow_field_name,
        wrf_hydro_channel_restart_downstream_flow_field_name,
        wrf_hydro_channel_restart_depth_flow_field_name,
    )
else:
    # TODO: Consider adding option to read cold state from route-link file
    channel_initial_us_flow_const = 0.0
    channel_initial_ds_flow_const = 0.0
    channel_initial_depth_const = 0.0
    # Set initial states from cold-state
    channel_initial_states_df = pd.DataFrame(
        0, index=connections.keys(), columns=["qu0", "qd0", "h0",], dtype="float32"
    )
    channel_initial_states_df["qu0"] = channel_initial_us_flow_const
    channel_initial_states_df["qd0"] = channel_initial_ds_flow_const
    channel_initial_states_df["h0"] = channel_initial_depth_const
    channel_initial_states_df["index"] = range(len(channel_initial_states_df))

if verbose:
    print("channel initial states complete")
if showtiming:
    print("... in %s seconds." % (time.time() - start_time))
    start_time = time.time()


In [ ]:
# STEP 5: Read (or set) QLateral Inputs
if showtiming:
    start_time = time.time()
if verbose:
    print("creating qlateral array ...")

# initialize qlateral dict
qlateral = {}

if qlat_input_folder:
    qlat_files = []
    for pattern in qlat_file_pattern_filter:
        qlat_files.extend(glob.glob(qlat_input_folder + pattern))
    qlat_df = nio.get_ql_from_wrf_hydro(
        qlat_files=qlat_files,
        index_col=qlat_file_index_col,
        value_col=qlat_file_value_col,
    )

elif qlat_input_file:
    qlat_df = nio.get_ql_from_csv(qlat_input_file)

else:
    qlat_df = pd.DataFrame(
        qlat_const, index=connections.keys(), columns=range(nts), dtype="float32"
    )

for index, row in qlat_df.iterrows():
    qlateral[index] = row.tolist()

if verbose:
    print("qlateral array complete")
if showtiming:
    print("... in %s seconds." % (time.time() - start_time))
    start_time = time.time()


In [ ]:
# STEP 6: Sort the ordered networks
if sort_networks:
    if showtiming:
        start_time = time.time()
    if verbose:
        print("sorting the ordered networks ...")

    for nsq in range(max_network_seqorder, -1, -1):
        sort_ordered_network(ordered_networks[nsq], True)

    if verbose:
        print("sorting complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))
        start_time = time.time()


In [ ]:
# Define the pool after we create the static global objects (and collect the garbage)
if parallel_compute:
    import gc

    gc.collect()
    pool = multiprocessing.Pool(cpu_pool)

flowveldepth_connect = (
    {}
)  # dict to contain values to transfer from upstream to downstream networks


In [ ]:
################### Main Execution Loop across ordered networks
if showtiming:
    main_start_time = time.time()
if verbose:
    print(f"executing routing computation ...")

compute_network_func = tr.compute_network

tr.connections_g = connections
tr.networks_g = networks
tr.qlateral_g = qlateral
tr.waterbodies_df_g = waterbodies_df
tr.waterbody_initial_states_df_g = waterbody_initial_states_df
tr.channel_initial_states_df_g = channel_initial_states_df

progress_count = 0
percentage_complete = True
if percentage_complete:
    for nsq in range(max_network_seqorder, -1, -1):
        for terminal_segment, network in ordered_networks[nsq]:
            progress_count += len(network["all_segments"])
    pbar = tqdm(total=(progress_count))

for nsq in range(max_network_seqorder, -1, -1):

    if parallel_compute:
        nslist = []
    results = []

    current_index_total = 0

    for terminal_segment, network in ordered_networks[nsq]:

        if percentage_complete:
            if current_index_total == 0:
                pbar.update(0)

        if break_network_at_waterbodies:
            waterbody = waterbodies_segments.get(terminal_segment)
        else:
            waterbody = None
        if not parallel_compute:  # serial execution
            if showtiming:
                start_time = time.time()
            if verbose:
                print(
                    f"routing ordered reaches for terminal segment {terminal_segment} ..."
                )

            results.append(
                compute_network_func(
                    flowveldepth_connect=flowveldepth_connect,
                    terminal_segment=terminal_segment,
                    supernetwork_parameters=supernetwork_parameters,
                    waterbody_parameters=waterbody_parameters,
                    waterbody=waterbody,
                    nts=nts,
                    dt=dt,
                    qts_subdivisions=qts_subdivisions,
                    verbose=verbose,
                    debuglevel=debuglevel,
                    csv_output=csv_output,
                    nc_output_folder=nc_output_folder,
                    assume_short_ts=assume_short_ts,
                )
            )

            if showtiming:
                print("... complete in %s seconds." % (time.time() - start_time))
            if percentage_complete:
                pbar.update(len(network["all_segments"]))

        else:  # parallel execution
            nslist.append(
                [
                    flowveldepth_connect,
                    terminal_segment,
                    supernetwork_parameters,  # TODO: This should probably be global...
                    waterbody_parameters,
                    waterbody,
                    nts,
                    dt,
                    qts_subdivisions,
                    verbose,
                    debuglevel,
                    csv_output,
                    nc_output_folder,
                    assume_short_ts,
                ]
            )

    if parallel_compute:
        if verbose:
            print(f"routing ordered reaches for networks of order {nsq} ... ")
        if debuglevel <= -2:
            print(f"reaches to be routed include:")
            print(f"{[network[0] for network in ordered_networks[nsq]]}")
        # with pool:
        # with multiprocessing.Pool() as pool:
        results = pool.starmap(compute_network_func, nslist)

        if showtiming:
            print("... complete in %s seconds." % (time.time() - start_time))
        if percentage_complete:
            pbar.update(
                sum(
                    len(network[1]["all_segments"]) for network in ordered_networks[nsq]
                )
            )
            # print(f"{[network[0] for network in ordered_networks[nsq]]}")

    max_courant = 0
    maxa = []
    for result in results:
        for seg in result:
            maxa.extend(result[seg][:, 8:9])
    max_courant = max(maxa)
    print(f"max_courant: {max_courant}")

    if (
        nsq > 0
    ):  # We skip this step for zero-order networks, i.e., those that have no downstream dependents
        flowveldepth_connect = (
            {}
        )  # There is no need to preserve previously passed on values -- so we clear the dictionary
        for i, (terminal_segment, network) in enumerate(ordered_networks[nsq]):
            # seg = network["reaches"][network["terminal_reach"]]["reach_tail"]
            seg = terminal_segment
            flowveldepth_connect[seg] = {}
            flowveldepth_connect[seg] = results[i][seg]
            # TODO: The value passed here could be much more specific to
            # TODO: exactly and only the most recent time step for the passing reach

if parallel_compute:
    pool.close()

if percentage_complete:
    pbar.close()

if verbose:
    print("ordered reach computation complete")
if showtiming:
    print("... in %s seconds." % (time.time() - main_start_time))
if verbose:
    print("program complete")
if showtiming:
    print("... in %s seconds." % (time.time() - program_start_time))


In [ ]:
# creation of the main dataframe


In [ ]:
# print(results)
import numpy as np
import itertools
import pandas as pd

all_results = {}
seg_courant_maxes = []
time = []
flowval = []  # flowval
velval_list = []  # velval
depthval = []  # depthval
qlatval = []  # qlatval
storageval = []  # storageval
qlatCumval = []  # qlatCumval
kinCelerity = []  # ck
courant = []  # cn
X = []  # X

for result in results:
    all_results.update(result)

# print(all_results[8780801][1][0])

df = pd.DataFrame()
# print(all_results)

for key, data in all_results.items():
    time.extend(data[:, 0])  # time
    flowval.extend(data[:, 1])  # flowval
    velval_list.extend(data[:, 2])  # velval
    depthval.extend(data[:, 3])  # depthval
    qlatval.extend(data[:, 4])  # qlatval
    storageval.extend(data[:, 5])  # storageval
    qlatCumval.extend(data[:, 6])  # qlatCumval
    kinCelerity.extend(data[:, 7])  # ck
    courant.extend(data[:, 8])  # cn
    X.extend(data[:, 9])  # X


In [ ]:
single_seg_length = len(all_results[933020089][:, 0])
single_seg_length * 611


In [ ]:
len(time)


In [ ]:
key_index = all_results.keys()
key_index = list(
    itertools.chain.from_iterable(
        itertools.repeat(x, single_seg_length) for x in key_index
    )
)
data = {
    "key_index": list(key_index),
    "time": list(time),
    "flowval": list(flowval),
    "velval_list": list(velval_list),
    "depthval": list(depthval),
    "qlatval": list(qlatval),
    "storageval": list(storageval),
    "qlatCumval": list(qlatCumval),
    "kinCelerity": list(kinCelerity),
    "courant": list(courant),
    "X": list(X),
}
df = pd.DataFrame(data)
df = df.set_index("key_index")
df = df.reset_index()
df = df.set_index("key_index")

df


In [ ]:
!pip install plotly

In [ ]:
# top 25 courants based on our work this morning and their segment IDs


In [ ]:
seg_list = []
seg_courant_maxes = []
for seg in all_results:
    seg_list.append(seg)
    seg_courant_maxes.append(max(all_results[seg][:, tr.courant_index]))
zipped = zip(seg_list, seg_courant_maxes)
zipped = list(zipped)
res = sorted(zipped, key=lambda x: x[1], reverse=True)
# A.sort(reverse=True)
res = (res)[:25]
print((res)[:25])


In [ ]:
# A list of those IDs


In [ ]:
major_segments = []
for x, y in res[:25]:
    major_segments.append(x)
major_segments


In [ ]:
# Filtering out the above segment IDs from the original df.


In [ ]:
df_filtered = df.loc[df.index.isin(major_segments), :]
df_filtered


In [ ]:
df.loc[df["courant"] == 25.363832473754883]


In [ ]:
# Automated plotter based on filtered dataframe


In [ ]:
import plotly.graph_objects as go
from plotly.offline import plot
import plotly.io as pio

df_indexed = df_filtered.reset_index()
test_chart = go.FigureWidget()
for x in range(0, 25):
    temp_df_range_1 = single_seg_length * (x)
    temp_df_range_2 = single_seg_length * (x + 1)
    print(temp_df_range_1, temp_df_range_2)
    test_chart.add_scatter(
        x=df_indexed[temp_df_range_1:temp_df_range_2]["time"],
        y=df_indexed[temp_df_range_1:temp_df_range_2]["flowval"],
        name="segment " + str(df_indexed["key_index"][temp_df_range_1]),
    )
test_chart.layout.title = "Timestep Chart " + str(dt)
plot(test_chart)


In [ ]:
custom_input_folder = root.joinpath("test", "input", "yaml")
custom_input_file = "florence_933020089_dt300.yaml"
all_results = tr.route_supernetwork(
    tr.connections_g,
    tr.networks_g,
    tr.qlateral_g,
    tr.waterbodies_df_g,
    tr.waterbody_initial_states_df_g,
    tr.channel_initial_states_df_g,
    custom_input_file=custom_input_folder.joinpath(custom_input_file),
)


In [118]:
custom_input_file_list = ["florence_933020089_dt300.yaml","florence_933020089_dt60.yaml"]
for i,c_i in enumerate(custom_input_file_list):
    print(i,c_i)
    import pathlib
    import sys
    import time
    import glob
    from tqdm import tqdm
    import multiprocessing

    root = pathlib.Path("../../").resolve()
    sys.path.append(str(root.joinpath("src", "python_framework_v02")))
    sys.path.append(str(root.joinpath("src", "python_framework_v01")))
    sys.path.append(str(root.joinpath("src", "python_routing_v01")))
    sys.path.append(".")
    import nhd_io as nio
    import compute_nhd_routing_SingleSeg as tr
    import nhd_network_utilities_v01 as nnu
    import nhd_reach_utilities as nru

    custom_input_folder = root.joinpath("test", "input", "yaml")
    custom_input_file = c_i
    run_pocono2_test = None

    supernetwork_parameters = None
    waterbody_parameters = None
    if custom_input_file:
        (
            supernetwork_parameters,
            waterbody_parameters,
            forcing_parameters,
            restart_parameters,
            output_parameters,
            run_parameters,
        ) = nio.read_custom_input(custom_input_folder.joinpath(custom_input_file))

        break_network_at_waterbodies = run_parameters.get(
            "break_network_at_waterbodies", None
        )

        dt = run_parameters.get("dt", None)
        nts = run_parameters.get("nts", None)
        qts_subdivisions = run_parameters.get("qts_subdivisions", None)
        debuglevel = -1 * int(run_parameters.get("debuglevel", 0))
        verbose = run_parameters.get("verbose", None)
        showtiming = run_parameters.get("showtiming", None)
        percentage_complete = run_parameters.get("percentage_complete", None)
        do_network_analysis_only = run_parameters.get("do_network_analysis_only", None)
        assume_short_ts = run_parameters.get("assume_short_ts", None)
        parallel_compute = run_parameters.get("parallel_compute", None)
        cpu_pool = run_parameters.get("cpu_pool", None)
        sort_networks = run_parameters.get("sort_networks", None)

        csv_output = output_parameters.get("csv_output", None)
        nc_output_folder = output_parameters.get("nc_output_folder", None)

        qlat_const = forcing_parameters.get("qlat_const", None)
        qlat_input_file = forcing_parameters.get("qlat_input_file", None)
        qlat_input_folder = forcing_parameters.get("qlat_input_folder", None)
        qlat_file_pattern_filter = forcing_parameters.get("qlat_file_pattern_filter", None)
        qlat_file_index_col = forcing_parameters.get("qlat_file_index_col", None)
        qlat_file_value_col = forcing_parameters.get("qlat_file_value_col", None)

        wrf_hydro_channel_restart_file = restart_parameters.get(
            "wrf_hydro_channel_restart_file", None
        )
        wrf_hydro_channel_ID_crosswalk_file = restart_parameters.get(
            "wrf_hydro_channel_ID_crosswalk_file", None
        )
        wrf_hydro_channel_ID_crosswalk_file_field_name = restart_parameters.get(
            "wrf_hydro_channel_ID_crosswalk_file_field_name", None
        )
        wrf_hydro_channel_restart_upstream_flow_field_name = restart_parameters.get(
            "wrf_hydro_channel_restart_upstream_flow_field_name", None
        )
        wrf_hydro_channel_restart_downstream_flow_field_name = restart_parameters.get(
            "wrf_hydro_channel_restart_downstream_flow_field_name", None
        )
        wrf_hydro_channel_restart_depth_flow_field_name = restart_parameters.get(
            "wrf_hydro_channel_restart_depth_flow_field_name", None
        )

        wrf_hydro_waterbody_restart_file = restart_parameters.get(
            "wrf_hydro_waterbody_restart_file", None
        )
        wrf_hydro_waterbody_ID_crosswalk_file = restart_parameters.get(
            "wrf_hydro_waterbody_ID_crosswalk_file", None
        )
        wrf_hydro_waterbody_ID_crosswalk_file_field_name = restart_parameters.get(
            "wrf_hydro_waterbody_ID_crosswalk_file_field_name", None
        )
        wrf_hydro_waterbody_crosswalk_filter_file = restart_parameters.get(
            "wrf_hydro_waterbody_crosswalk_filter_file", None
        )
        wrf_hydro_waterbody_crosswalk_filter_file_field_name = restart_parameters.get(
            "wrf_hydro_waterbody_crosswalk_filter_file_field_name", None
        )

    # Any specific commandline arguments will override the file
    # TODO: There are probably some pathological collisions that could
    # arise from this ordering ... check these out.

    if run_pocono2_test:
        if verbose:
            print("running test case for Pocono_TEST2 domain")
        # Overwrite the following test defaults
        supernetwork = "Pocono_TEST2"
        break_network_at_waterbodies = False
        qts_subdivisions = 1  # change qts_subdivisions = 1 as  default
        dt = 300 / qts_subdivisions
        nts = 144 * qts_subdivisions
        csv_output = {"csv_output_folder": os.path.join(root, "test", "output", "text")}
        nc_output_folder = os.path.join(root, "test", "output", "text")
        # test 1. Take lateral flow from re-formatted wrf-hydro output from Pocono Basin simulation
        qlat_input_file = os.path.join(
            root, r"test/input/geo/PoconoSampleData2/Pocono_ql_testsamp1_nwm_mc.csv"
        )
    if showtiming:
        program_start_time = time.time()
    if verbose:
        print(f"begin program t-route ...")

    # STEP 1: Read the supernetwork dataset and build the connections graph
    if verbose:
        print("creating supernetwork connections set")
    if showtiming:
        start_time = time.time()

    if supernetwork_parameters:
        supernetwork_values = nnu.get_nhd_connections(
            supernetwork_parameters=supernetwork_parameters,
            verbose=False,
            debuglevel=debuglevel,
        )

    else:
        test_folder = os.path.join(root, r"test")
        geo_input_folder = os.path.join(test_folder, r"input", r"geo")
        supernetwork_parameters, supernetwork_values = nnu.set_networks(
            supernetwork=supernetwork,
            geo_input_folder=geo_input_folder,
            verbose=False,
            debuglevel=debuglevel,
        )
        waterbody_parameters = nnu.set_waterbody_parameters(
            supernetwork=supernetwork,
            geo_input_folder=geo_input_folder,
            verbose=False,
            debuglevel=debuglevel,
        )

    if verbose:
        print("supernetwork connections set complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))

    connections = supernetwork_values[0]
    # STEP 2: Separate the networks and build the sub-graph of reaches within each network
    if showtiming:
        start_time = time.time()
    if verbose:
        print("organizing connections into reaches ...")
    networks = nru.compose_networks(
        supernetwork_values,
        break_network_at_waterbodies=break_network_at_waterbodies,
        verbose=False,
        debuglevel=debuglevel,
        showtiming=showtiming,
    )

    if verbose:
        print("reach organization complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))
        start_time = time.time()
    # STEP 3: Organize Network for Waterbodies
    if break_network_at_waterbodies:
        if showtiming:
            start_time = time.time()
        if verbose:
            print("reading waterbody parameter file ...")

        ## STEP 3a: Read waterbody parameter file
        waterbodies_values = supernetwork_values[12]
        waterbodies_segments = supernetwork_values[13]
        connections_tailwaters = supernetwork_values[4]

        waterbodies_df = nio.read_waterbody_df(waterbody_parameters, waterbodies_values,)
        waterbodies_df = waterbodies_df.sort_index(axis="index").sort_index(axis="columns")

        nru.order_networks(connections, networks, connections_tailwaters)

        if verbose:
            print("waterbodies complete")
        if showtiming:
            print("... in %s seconds." % (time.time() - start_time))
            start_time = time.time()

        ## STEP 3b: Order subnetworks above and below reservoirs
        if showtiming:
            start_time = time.time()
        if verbose:
            print("ordering waterbody subnetworks ...")

        max_network_seqorder = -1
        for network in networks:
            max_network_seqorder = max(
                networks[network]["network_seqorder"], max_network_seqorder
            )
        ordered_networks = {}

        for terminal_segment, network in networks.items():
            if network["network_seqorder"] not in ordered_networks:
                ordered_networks[network["network_seqorder"]] = []
            ordered_networks[network["network_seqorder"]].append(
                (terminal_segment, network)
            )

        if verbose:
            print("ordering waterbody subnetworks complete")
        if showtiming:
            print("... in %s seconds." % (time.time() - start_time))
            start_time = time.time()

    else:
        # If we are not splitting the networks, we can put them all in one order
        max_network_seqorder = 0
        ordered_networks = {}
        ordered_networks[0] = [
            (terminal_segment, network) for terminal_segment, network in networks.items()
        ]

    if do_network_analysis_only:
        sys.exit()

    if break_network_at_waterbodies:
        ## STEP 3c: Handle Waterbody Initial States
        if showtiming:
            start_time = time.time()
        if verbose:
            print("setting waterbody initial states ...")

        if wrf_hydro_waterbody_restart_file:

            waterbody_initial_states_df = nio.get_reservoir_restart_from_wrf_hydro(
                wrf_hydro_waterbody_restart_file,
                wrf_hydro_waterbody_ID_crosswalk_file,
                wrf_hydro_waterbody_ID_crosswalk_file_field_name,
                wrf_hydro_waterbody_crosswalk_filter_file,
                wrf_hydro_waterbody_crosswalk_filter_file_field_name,
            )
        else:
            # TODO: Consider adding option to read cold state from route-link file
            waterbody_initial_ds_flow_const = 0.0
            waterbody_initial_depth_const = 0.0
            # Set initial states from cold-state
            waterbody_initial_states_df = pd.DataFrame(
                0, index=waterbodies_df.index, columns=["qd0", "h0",], dtype="float32"
            )
            # TODO: This assignment could probably by done in the above call
            waterbody_initial_states_df["qd0"] = waterbody_initial_ds_flow_const
            waterbody_initial_states_df["h0"] = waterbody_initial_depth_const
            waterbody_initial_states_df["index"] = range(len(waterbody_initial_states_df))

        if verbose:
            print("waterbody initial states complete")
        if showtiming:
            print("... in %s seconds." % (time.time() - start_time))
            start_time = time.time()
    # STEP 4: Handle Channel Initial States
    if showtiming:
        start_time = time.time()
    if verbose:
        print("setting channel initial states ...")

    if wrf_hydro_channel_restart_file:

        channel_initial_states_df = nio.get_stream_restart_from_wrf_hydro(
            wrf_hydro_channel_restart_file,
            wrf_hydro_channel_ID_crosswalk_file,
            wrf_hydro_channel_ID_crosswalk_file_field_name,
            wrf_hydro_channel_restart_upstream_flow_field_name,
            wrf_hydro_channel_restart_downstream_flow_field_name,
            wrf_hydro_channel_restart_depth_flow_field_name,
        )
    else:
        # TODO: Consider adding option to read cold state from route-link file
        channel_initial_us_flow_const = 0.0
        channel_initial_ds_flow_const = 0.0
        channel_initial_depth_const = 0.0
        # Set initial states from cold-state
        channel_initial_states_df = pd.DataFrame(
            0, index=connections.keys(), columns=["qu0", "qd0", "h0",], dtype="float32"
        )
        channel_initial_states_df["qu0"] = channel_initial_us_flow_const
        channel_initial_states_df["qd0"] = channel_initial_ds_flow_const
        channel_initial_states_df["h0"] = channel_initial_depth_const
        channel_initial_states_df["index"] = range(len(channel_initial_states_df))

    if verbose:
        print("channel initial states complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))
        start_time = time.time()
    # STEP 5: Read (or set) QLateral Inputs
    if showtiming:
        start_time = time.time()
    if verbose:
        print("creating qlateral array ...")

    # initialize qlateral dict
    qlateral = {}

    if qlat_input_folder:
        qlat_files = []
        for pattern in qlat_file_pattern_filter:
            qlat_files.extend(glob.glob(qlat_input_folder + pattern))
        qlat_df = nio.get_ql_from_wrf_hydro(
            qlat_files=qlat_files,
            index_col=qlat_file_index_col,
            value_col=qlat_file_value_col,
        )

    elif qlat_input_file:
        qlat_df = nio.get_ql_from_csv(qlat_input_file)

    else:
        qlat_df = pd.DataFrame(
            qlat_const, index=connections.keys(), columns=range(nts), dtype="float32"
        )

    for index, row in qlat_df.iterrows():
        qlateral[index] = row.tolist()

    if verbose:
        print("qlateral array complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))
        start_time = time.time()
    # STEP 6: Sort the ordered networks
    if sort_networks:
        if showtiming:
            start_time = time.time()
        if verbose:
            print("sorting the ordered networks ...")

        for nsq in range(max_network_seqorder, -1, -1):
            sort_ordered_network(ordered_networks[nsq], True)

        if verbose:
            print("sorting complete")
        if showtiming:
            print("... in %s seconds." % (time.time() - start_time))
            start_time = time.time()
    # Define the pool after we create the static global objects (and collect the garbage)
    if parallel_compute:
        import gc

        gc.collect()
        pool = multiprocessing.Pool(cpu_pool)

    flowveldepth_connect = (
        {}
    )  # dict to contain values to transfer from upstream to downstream networks
    ################### Main Execution Loop across ordered networks
    if showtiming:
        main_start_time = time.time()
    if verbose:
        print(f"executing routing computation ...")

    compute_network_func = tr.compute_network

    tr.connections_g = connections
    tr.networks_g = networks
    tr.qlateral_g = qlateral
    tr.waterbodies_df_g = waterbodies_df
    tr.waterbody_initial_states_df_g = waterbody_initial_states_df
    tr.channel_initial_states_df_g = channel_initial_states_df

    progress_count = 0
    percentage_complete = True
    if percentage_complete:
        for nsq in range(max_network_seqorder, -1, -1):
            for terminal_segment, network in ordered_networks[nsq]:
                progress_count += len(network["all_segments"])
        pbar = tqdm(total=(progress_count))

    for nsq in range(max_network_seqorder, -1, -1):

        if parallel_compute:
            nslist = []
        results = []

        current_index_total = 0

        for terminal_segment, network in ordered_networks[nsq]:

            if percentage_complete:
                if current_index_total == 0:
                    pbar.update(0)

            if break_network_at_waterbodies:
                waterbody = waterbodies_segments.get(terminal_segment)
            else:
                waterbody = None
            if not parallel_compute:  # serial execution
                if showtiming:
                    start_time = time.time()
                if verbose:
                    print(
                        f"routing ordered reaches for terminal segment {terminal_segment} ..."
                    )

                results.append(
                    compute_network_func(
                        flowveldepth_connect=flowveldepth_connect,
                        terminal_segment=terminal_segment,
                        supernetwork_parameters=supernetwork_parameters,
                        waterbody_parameters=waterbody_parameters,
                        waterbody=waterbody,
                        nts=nts,
                        dt=dt,
                        qts_subdivisions=qts_subdivisions,
                        verbose=verbose,
                        debuglevel=debuglevel,
                        csv_output=csv_output,
                        nc_output_folder=nc_output_folder,
                        assume_short_ts=assume_short_ts,
                    )
                )

                if showtiming:
                    print("... complete in %s seconds." % (time.time() - start_time))
                if percentage_complete:
                    pbar.update(len(network["all_segments"]))

            else:  # parallel execution
                nslist.append(
                    [
                        flowveldepth_connect,
                        terminal_segment,
                        supernetwork_parameters,  # TODO: This should probably be global...
                        waterbody_parameters,
                        waterbody,
                        nts,
                        dt,
                        qts_subdivisions,
                        verbose,
                        debuglevel,
                        csv_output,
                        nc_output_folder,
                        assume_short_ts,
                    ]
                )

        if parallel_compute:
            if verbose:
                print(f"routing ordered reaches for networks of order {nsq} ... ")
            if debuglevel <= -2:
                print(f"reaches to be routed include:")
                print(f"{[network[0] for network in ordered_networks[nsq]]}")
            # with pool:
            # with multiprocessing.Pool() as pool:
            results = pool.starmap(compute_network_func, nslist)

            if showtiming:
                print("... complete in %s seconds." % (time.time() - start_time))
            if percentage_complete:
                pbar.update(
                    sum(
                        len(network[1]["all_segments"]) for network in ordered_networks[nsq]
                    )
                )
                # print(f"{[network[0] for network in ordered_networks[nsq]]}")

        max_courant = 0
        maxa = []
        for result in results:
            for seg in result:
                maxa.extend(result[seg][:, 8:9])
        max_courant = max(maxa)
        print(f"max_courant: {max_courant}")

        if (
            nsq > 0
        ):  # We skip this step for zero-order networks, i.e., those that have no downstream dependents
            flowveldepth_connect = (
                {}
            )  # There is no need to preserve previously passed on values -- so we clear the dictionary
            for i, (terminal_segment, network) in enumerate(ordered_networks[nsq]):
                # seg = network["reaches"][network["terminal_reach"]]["reach_tail"]
                seg = terminal_segment
                flowveldepth_connect[seg] = {}
                flowveldepth_connect[seg] = results[i][seg]
                # TODO: The value passed here could be much more specific to
                # TODO: exactly and only the most recent time step for the passing reach

    if parallel_compute:
        pool.close()

    if percentage_complete:
        pbar.close()

    if verbose:
        print("ordered reach computation complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - main_start_time))
    if verbose:
        print("program complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - program_start_time))
    
    
    
    all_results_new = {}
    seg_courant_maxes = []
    time = []
    flowval = []  # flowval
    velval_list = []  # velval
    depthval = []  # depthval
    qlatval = []  # qlatval
    storageval = []  # storageval
    qlatCumval = []  # qlatCumval
    kinCelerity = []  # ck
    courant = []  # cn
    X = []  # X

    for result in results:
        all_results_new.update(result)

    # print(all_results[8780801][1][0])

    df = pd.DataFrame()
    # print(all_results)

    for key, data in all_results_new.items():
        time.extend(data[:, 0])  # time
        flowval.extend(data[:, 1])  # flowval
        velval_list.extend(data[:, 2])  # velval
        depthval.extend(data[:, 3])  # depthval
        qlatval.extend(data[:, 4])  # qlatval
        storageval.extend(data[:, 5])  # storageval
        qlatCumval.extend(data[:, 6])  # qlatCumval
        kinCelerity.extend(data[:, 7])  # ck
        courant.extend(data[:, 8])  # cn
        X.extend(data[:, 9])  # X
#     print(all_results_new)
    single_seg_length = len(all_results_new[933020089][:, 0])
    print(single_seg_length)
    print(i)
    if i == 0:
        single_seg_length_300 = single_seg_length
    elif i == 1:
        single_seg_length_60 = single_seg_length
#     else:
#         single_seg_length_10 = single_seg_length

    # single_seg_length * 611

    key_index = all_results.keys()
    key_index = list(
        itertools.chain.from_iterable(
            itertools.repeat(x, single_seg_length) for x in key_index
        )
    )
    data = {
        "key_index": list(key_index),
        "time": list(time),
        "flowval": list(flowval),
        "velval_list": list(velval_list),
        "depthval": list(depthval),
        "qlatval": list(qlatval),
        "storageval": list(storageval),
        "qlatCumval": list(qlatCumval),
        "kinCelerity": list(kinCelerity),
        "courant": list(courant),
        "X": list(X),
    }
    df = pd.DataFrame(data)
    df = df.set_index("key_index")
    df = df.reset_index()
    df = df.set_index("key_index")


    seg_list = []
    seg_courant_maxes = []
    for seg in all_results_new:
        seg_list.append(seg)
        seg_courant_maxes.append(max(all_results_new[seg][:, tr.courant_index]))
    zipped = zip(seg_list, seg_courant_maxes)
    zipped = list(zipped)
    res = sorted(zipped, key=lambda x: x[1], reverse=True)
    # A.sort(reverse=True)
    res = (res)[:3]
    # print((res)[:25])

    major_segments = []
    for x, y in res[:3]:
        major_segments.append(x)
    # major_segments

    df_filtered = df.loc[df.index.isin(major_segments), :]

    if i == 0:
        df_indexed_300 = df_filtered.reset_index()
    elif i == 1:
        df_indexed_60 = df_filtered.reset_index()
#     else:
#         df_indexed_10 = df_filtered.reset_index()

test_chart = go.FigureWidget()
for x in range(0,3):
    temp_df_range_1 = single_seg_length_300 * (x)
    temp_df_range_2 = single_seg_length_300 * (x + 1)
    #         print(temp_df_range_1, temp_df_range_2)
    test_chart.add_scatter(
        x=df_indexed_300[temp_df_range_1:temp_df_range_2]["time"],
        y=df_indexed_300[temp_df_range_1:temp_df_range_2]["flowval"],
        name="segment " + str(df_indexed_300["key_index"][temp_df_range_1]) + " " + str(300),
    )
    temp_df_range_1 = single_seg_length_60 * (x)
    temp_df_range_2 = single_seg_length_60 * (x + 1)
    #         print(temp_df_range_1, temp_df_range_2)
    test_chart.add_scatter(
        x=df_indexed_60[temp_df_range_1:temp_df_range_2]["time"],
        y=df_indexed_60[temp_df_range_1:temp_df_range_2]["flowval"],
        name="segment " + str(df_indexed_60["key_index"][temp_df_range_1]) + " " + str(60),
    )
    #     temp_df_range_1 = single_seg_length_10 * (x)
    #     temp_df_range_2 = single_seg_length_10 * (x + 1)
    # #         print(temp_df_range_1, temp_df_range_2)
    #     test_chart.add_scatter(
    #         x=df_indexed_10[temp_df_range_1:temp_df_range_2]["time"],
    #         y=df_indexed_10[temp_df_range_1:temp_df_range_2]["flowval"],
    #         name="segment " + str(df_indexed_10["key_index"][temp_df_range_1]) + " " + str(dt),
    #     )

test_chart.layout.title = "Timestep Chart 300, 60"
plot(test_chart)



 42%|████▏     | 467/1108 [00:06<00:08, 77.75it/s]

... complete in 7.442021131515503 seconds.
max_courant: [27.53112221]
routing ordered reaches for networks of order 1 ... 


 45%|████▌     | 499/1108 [00:09<00:24, 25.35it/s]

... complete in 10.689549922943115 seconds.
max_courant: [0.]
routing ordered reaches for networks of order 0 ... 
writing segment output to --> ../../test/output/text/933020089.csv


100%|██████████| 1108/1108 [00:19<00:00, 30.74it/s]

...

... complete in 93.49365997314453 seconds.


100%|██████████| 1108/1108 [01:35<00:00, 11.59it/s]

max_courant: [25.36383247]
ordered reach computation complete
... in 95.66082572937012 seconds.
program complete
... in 102.13372921943665 seconds.


10080
1


KeyError: 'time'

In [ ]:
single_seg_length_300

In [ ]:
df_indexed_300

In [ ]:
custom_input_file_list = ["florence_933020089_dt300.yaml","florence_933020089_dt60.yaml"]
# custom_input_file_list = ["florence_933020089_dt300.yaml"]
df_indexed_300 = pd.DataFrame()
df_indexed_60 = pd.DataFrame()
# df_indexed_10 = pd.DataFrame()
single_seg_length_300 = 0
single_seg_length_60 = 0
# single_seg_length_10 = 0
for i,c_i in enumerate(custom_input_file_list):
    print(c_i)
    custom_input_folder = root.joinpath("test", "input", "yaml")
    custom_input_file = c_i
    all_results_new = tr.route_supernetwork(
        tr.connections_g,
        tr.networks_g,
        tr.qlateral_g,
        tr.waterbodies_df_g,
        tr.waterbody_initial_states_df_g,
        tr.channel_initial_states_df_g,
        custom_input_file=custom_input_folder.joinpath(custom_input_file),
    )
    

    
    print(len(all_results_new))
    all_results_new = {}
    seg_courant_maxes = []
    time = []
    flowval = []  # flowval
    velval_list = []  # velval
    depthval = []  # depthval
    qlatval = []  # qlatval
    storageval = []  # storageval
    qlatCumval = []  # qlatCumval
    kinCelerity = []  # ck
    courant = []  # cn
    X = []  # X

    for result in results:
        all_results_new.update(result)

    # print(all_results[8780801][1][0])

    df = pd.DataFrame()
    # print(all_results)

    for key, data in all_results_new.items():
        time.extend(data[:, 0])  # time
        flowval.extend(data[:, 1])  # flowval
        velval_list.extend(data[:, 2])  # velval
        depthval.extend(data[:, 3])  # depthval
        qlatval.extend(data[:, 4])  # qlatval
        storageval.extend(data[:, 5])  # storageval
        qlatCumval.extend(data[:, 6])  # qlatCumval
        kinCelerity.extend(data[:, 7])  # ck
        courant.extend(data[:, 8])  # cn
        X.extend(data[:, 9])  # X
#     print(all_results_new)
    single_seg_length = len(all_results_new[933020089][:, 0])
    print(single_seg_length)
    if i == 0:
        single_seg_length_300 = single_seg_length
    elif i == 1:
        single_seg_length_60 = single_seg_length
#     else:
#         single_seg_length_10 = single_seg_length

    # single_seg_length * 611

    key_index = all_results.keys()
    key_index = list(
        itertools.chain.from_iterable(
            itertools.repeat(x, single_seg_length) for x in key_index
        )
    )
    data = {
        "key_index": list(key_index),
        "time": list(time),
        "flowval": list(flowval),
        "velval_list": list(velval_list),
        "depthval": list(depthval),
        "qlatval": list(qlatval),
        "storageval": list(storageval),
        "qlatCumval": list(qlatCumval),
        "kinCelerity": list(kinCelerity),
        "courant": list(courant),
        "X": list(X),
    }
    df = pd.DataFrame(data)
    df = df.set_index("key_index")
    df = df.reset_index()
    df = df.set_index("key_index")


    seg_list = []
    seg_courant_maxes = []
    for seg in all_results_new:
        seg_list.append(seg)
        seg_courant_maxes.append(max(all_results_new[seg][:, tr.courant_index]))
    zipped = zip(seg_list, seg_courant_maxes)
    zipped = list(zipped)
    res = sorted(zipped, key=lambda x: x[1], reverse=True)
    # A.sort(reverse=True)
    res = (res)[:3]
    # print((res)[:25])

    major_segments = []
    for x, y in res[:3]:
        major_segments.append(x)
    # major_segments

    df_filtered = df.loc[df.index.isin(major_segments), :]

    if i == 0:
        df_indexed_300 = df_filtered.reset_index()
    elif i == 1:
        df_indexed_60 = df_filtered.reset_index()
#     else:
#         df_indexed_10 = df_filtered.reset_index()

test_chart = go.FigureWidget()
for x in range(0,3):
    temp_df_range_1 = single_seg_length_300 * (x)
    temp_df_range_2 = single_seg_length_300 * (x + 1)
    #         print(temp_df_range_1, temp_df_range_2)
    test_chart.add_scatter(
        x=df_indexed_300[temp_df_range_1:temp_df_range_2]["time"],
        y=df_indexed_300[temp_df_range_1:temp_df_range_2]["flowval"],
        name="segment " + str(df_indexed_300["key_index"][temp_df_range_1]) + " " + str(300),
    )
    temp_df_range_1 = single_seg_length_60 * (x)
    temp_df_range_2 = single_seg_length_60 * (x + 1)
    #         print(temp_df_range_1, temp_df_range_2)
    test_chart.add_scatter(
        x=df_indexed_60[temp_df_range_1:temp_df_range_2]["time"],
        y=df_indexed_60[temp_df_range_1:temp_df_range_2]["flowval"],
        name="segment " + str(df_indexed_60["key_index"][temp_df_range_1]) + " " + str(60),
    )
    #     temp_df_range_1 = single_seg_length_10 * (x)
    #     temp_df_range_2 = single_seg_length_10 * (x + 1)
    # #         print(temp_df_range_1, temp_df_range_2)
    #     test_chart.add_scatter(
    #         x=df_indexed_10[temp_df_range_1:temp_df_range_2]["time"],
    #         y=df_indexed_10[temp_df_range_1:temp_df_range_2]["flowval"],
    #         name="segment " + str(df_indexed_10["key_index"][temp_df_range_1]) + " " + str(dt),
    #     )

test_chart.layout.title = "Timestep Chart 300, 60"
plot(test_chart)


In [ ]:
df_indexed_300 
# df_indexed_60 
# df_indexed_10 
single_seg_length_300
# single_seg_length_60 
# single_seg_length_10 

In [ ]:
single_seg_length_300

In [ ]:
single_seg_length_60